<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Custom_medical_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU langchain openai tiktoken duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [29]:
import langchain
from langchain.prompts.base import StringPromptTemplate
from langchain.prompts import PromptTemplate,StringPromptTemplate
from langchain.agents import Tool, AgentExecutor, AgentOutputParser,LLMSingleActionAgent,initialize_agent
from langchain import OpenAI,LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun
from langchain.schema import AgentAction,AgentFinish
import re
from typing import List,Union
import os
from getpass import getpass

In [7]:
os.environ["OPENAI_API_KEY"] = getpass()

··········


In [9]:
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    )
]

In [10]:
obj = search.run("How can I treat a sprained ankle?")
print(type(obj))
print(obj)

<class 'str'>
Sprained ankle — Injury to a ligament of the ankle can usually be treated with at-home care and appropriate exercises to get you back on your feet. This content does not have an English version. This content does not have an Arabic version. Skip to content Care at Mayo Clinic Care at Mayo Clinic About Mayo Clinic Request Appointment Find a Doctor Sprains - Diagnosis and treatment - Mayo Clinic A sprain is a stretching or tearing of ligaments — the tough bands of tissue that connect two bones together in your joints. Ice and elevation can help. This content does not have an English version. This content does not have an Arabic version. Skip to content Care at Mayo Clinic A sprain is a stretching or tearing of ligaments — the tough bands of fibrous tissue that connect two bones together in your joints. The most common location for a sprain is in your ankle. Initial treatment includes rest, ice, compression and elevation. Mild sprains can be successfully treated at home. Her

In [11]:
search.run("site:webmd.com How can I treat a sprained ankle?")

"RICE. 2 /14. It stands for rest, ice, compress, and elevate, and it's the go-to treatment for minor sprains and bruising. Ice the injury as soon as possible and put pressure on it with an ... Seek medical help for burns if they are severe; on the face, hands, feet, or genitals; or bigger than 2 inches. For treating small minor burns at home, rinse the area under running cool water ... You treat it the same way: Rest it, ice it, raise it, and use pain relievers. When to Call the Doctor . 8/13 . A bruise can need medical care if: You think a sprain or broken bone may have caused it. Treatment of myositis varies according to the cause. Causes of Myositis. Myositis is caused by any condition that leads to inflammation in the muscles. Myositis causes can be divided into several ... Being overweight. Stress fractures in your toes or metatarsals. Bunions. Torn ligaments in your feet. A condition known as Morton's neuroma (interdigital neuroma) also causes metatarsalgia-like ..."

In [12]:
def duck_wrapper(input_text):
    search_results = search.run(f"site:webmd.com {input_text}")
    return search_results

tools = [
    Tool(
        name = "Search WebMD",
        func=duck_wrapper,
        description="useful for when you need to answer medical and pharmalogical questions"
    )
]

## Prompt Template
This instructs the agent on what to do. Generally, the template should incorporate:

**`tools:`** which tools the agent has access and how and when to call them.

**`intermediate_steps:`** These are tuples of previous (**`AgentAction`**, **`Observation`**) pairs. These are generally not passed directly to the model, but the prompt template formats them in a specific way.

**`input:`** generic user input

In [21]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Question: {input}
"""

In [22]:
tool_names = [tool.name for tool in tools]

prompt = PromptTemplate(template=template,input_variables=["tools","tool_names","input"])

In [23]:
prompt

PromptTemplate(input_variables=['tools', 'tool_names', 'input'], output_parser=None, partial_variables={}, template='Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin! Remember to answer as a compansionate medical professional when giving your final answer.\n\nQuestion: {input}\n', template_format='f-string', validate_template=True)

In [24]:
llm= OpenAI()

In [30]:
from langchain.agents.agent_types import AgentType
agent = initialize_agent(tools=tools,
                         llm=ChatOpenAI(),
                         agent = AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                         verbose=True,
                         )

In [33]:
agent.run({"input":"how to treat acid reflux?","tools":tools,"tool_names":tool_names,"chat_history":None})



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Search WebMD
Action Input: "how to treat acid reflux"
Observation: Eat Earlier. Going to bed on a full stomach makes nighttime heartburn more likely. A full stomach puts pressure on the valve at the top of the stomach, which is supposed to keep stomach acid out ... Esophagitis Symptoms. Symptoms of esophagitis include: Difficult or painful swallowing. Acid reflux. Heartburn. A feeling of something of being stuck in the throat. Chest pain. Nausea. Vomiting. Abdominal bloating. Abdominal pain. Vomiting. Indigestion. Burning or gnawing feeling in the stomach between meals or at night. Hiccups. Loss of appetite. Vomiting blood or coffee ground-like ... Trim the fat off of meat and poultry, and cut the skin off chicken. Tweaks like these might be enough to tame your heartburn. Tomatoes (including foods like salsa and marinara sauce) and citrus ... This backflow (acid reflux) can irritate the lining of the esophagus. Pe

"To treat acid reflux, there are several lifestyle changes you can make such as eating smaller, more frequent meals; avoiding trigger foods such as spicy or acidic foods; and not lying down for at least three hours after eating. Over-the-counter medications such as antacids and acid reducers can also be helpful. However, if symptoms persist, it's important to consult with a healthcare professional for further evaluation and treatment options."

In [34]:
agent.run({"input":"How can I treat a sprained ankle?","tools":tools,"tool_names":tool_names,"chat_history":None})



> Entering new  chain...
Thought: Do I need to use a tool? Yes
Action: Search WebMD
Action Input: How to treat a sprained ankle
Observation: RICE. 2 /14. It stands for rest, ice, compress, and elevate, and it's the go-to treatment for minor sprains and bruising. Ice the injury as soon as possible and put pressure on it with an ... Seek medical help for burns if they are severe; on the face, hands, feet, or genitals; or bigger than 2 inches. For treating small minor burns at home, rinse the area under running cool water ... Being overweight. Stress fractures in your toes or metatarsals. Bunions. Torn ligaments in your feet. A condition known as Morton's neuroma (interdigital neuroma) also causes metatarsalgia-like ... You treat it the same way: Rest it, ice it, raise it, and use pain relievers. When to Call the Doctor . 8/13 . A bruise can need medical care if: You think a sprain or broken bone may have caused it. Treatment of myositis varies according to the cause. Causes of Myositis

"To treat a sprained ankle, you can use the RICE method: rest, ice, compress, and elevate. Rest the ankle, ice it as soon as possible, compress it with an elastic bandage, and elevate it to reduce swelling. You can also take over-the-counter pain relievers and avoid activities that may aggravate the injury. If the sprain is severe or does not improve with self-care measures, it's important to seek medical attention."